***
## Объект пользователя и проверка аутентификации

В объекте запроса `request` Django хранит множество полезной информации: URL и метод запроса, параметры запроса, содержимое заголовков headers и многое другое. В этом списке есть и объект `user` — это экземпляр модели пользователя, представляющий пользователя, отправившего запрос. 

Иными словами: если посетитель сайта аутентифицировался под логином и паролем, которые соответствуют объекту модели пользователя с id=38, то при любом запросе этого посетителя в объекте `request` будет передаваться объект модели `User` с id=38. Получить объект пользователя можно через точечную нотацию: `request.user`; так же через точечную нотацию доступны все поля этого объекта.

Если же запрос отправлен неаутентифицированным пользователем — объект `request.user` всё равно доступен; никакого id у него нет, зато атрибут `user.is_authenticated` содержит значение `False`.

Объект пользователя передаётся и в шаблон: сейчас в header.html уже применяется свойство `user.username` для вывода на страницу имени пользователя и `user.is_authenticated` — для отображения части навигации.

Именно атрибут `user.is_authenticated` и поможет организовать проверку «разрешено ли пользователю зайти на определённую страницу», например на страницу с формой для создания новой записи.

В проекте, где применяются view-функции, проверка может быть такой:

```py
from django.shortcuts import redirect
...

def birthday_create(request):
    if not request.user.is_authenticated:
        # Если пользователь не залогинен — отправляем его на страницу для входа:
        return redirect('login')
    # Если пользователь авторизован — выполняем полезный код функции.
    ... 
```

Код, выполняющий эту проверку, должен быть добавлен в каждую view-функцию, где требуется разграничить права доступа; таких функций может быть достаточно много. Повторение одного и того же кода нарушает принцип DRY и усложняет внесение изменений: при рефакторинге придётся повсюду править однотипный код.

Оптимальным решением здесь будет применение декораторов.

> **Декоратор** — это паттерн проектирования, предназначенный для расширения функциональности объектов без вмешательства в их код. Декоратор берёт функцию, ничего в ней не меняет, но добавляет к ней заданные действия. Одним декоратором можно «оборачивать» разные функции.

В Django есть встроенные декораторы, предназначенные для решения самых востребованных задач (а задача ограничения доступа для незалогиненных пользователей — как раз из таких).

Доступ к определённым страницам ограничивается на уровне представлений (views). Если запрошенная страница управляется view-функцией — перед объявлением этой функции добавляется встроенный декоратор @login_required; если страница генерируется с помощью CBV — к классу добавляется миксин `LoginRequiredMixin`. 

Декоратор `@login_required` и миксин `LoginRequiredMixin` добавляют в представление проверку: «залогинен ли пользователь?». Анонимных пользователей представление переадресует на стандартную страницу логина (адрес этой страницы указан в settings.py в константе `LOGIN_URL`), а аутентифицированному пользователю вернёт запрошенную страницу.

***
## Декоратор @login_required

Чтобы предоставить доступ к странице только залогиненным пользователям, достаточно «обернуть» view-функцию декоратором [@login_required](https://docs.djangoproject.com/en/3.2/topics/auth/default/#the-login-required-decorator); он импортируется из модуля `django.contrib.auth.decorators`.

Проверим работу декоратора на примере небольшой view-функции, которая возвращает текстовый ответ только аутентифицированным пользователям.

В файл *birthday/views.py* добавим view-функцию:

```py
# birthday/views.py
...
from django.contrib.auth.decorators import login_required
from django.http import HttpResponse


@login_required
def simple_view(request):
    return HttpResponse('Страница для залогиненных пользователей!')

... 
```

Настроим маршрут для новой страницы: в файле *birthday/urls.py* добавим новый путь в `urlpatterns`:

```py
# birthday/urls.py
...
urlpatterns = [
    ...
    path('login_only/', views.simple_view),
    ...
] 
```

***
## Проверка аутентификации для CBV

Использовать декоратор `@login_required` для CBV не получится.

```py
@login_required
class BirthdayCreateView(CreateView):
    model = Birthday
    form_class = BirthdayForm 
```

При запуске проекта Django выдаст ошибку:

```
AttributeError: 'function' object has no attribute 'as_view' 
```

Логично: декоратор всегда возвращает функцию, а в приведённом примере декоратор фактически подменил класс на функцию. В `urlpatterns` вызывается метод CBV `as_view()`:

```py
urlpatterns = [
    ...
    path('create/', views.BirthdayCreateView.as_view(), name='create'),
    ...
] 
```

Получается, Django должен вызвать метод `as_view()` у функции; получается полная ерунда: у функций нет методов! Вот всё и сломалось.

Можно добавить функцию-декоратор прямо в `urlpatterns`, и это будет работать:

```py
from django.contrib.auth.decorators import login_required

urlpatterns = [
    ...
    # Декорируем вызов метода as_view(), без синтаксического сахара:
    path('create/', login_required(views.BirthdayCreateView.as_view()), name='create'),
    ...
] 
```

Здесь маршрут `'create/'` связан с функцией-декоратором `login_required()`, которая декорирует метод `BirthdayCreateView.as_view()`.

Но этот способ не сильно распространён. Как правило, для классов применяют [миксин LoginRequiredMixin](https://docs.djangoproject.com/en/3.2/topics/auth/default/#the-loginrequired-mixin). Он добавляет в CBV проверку — аутентифицирован ли пользователь, сделавший запрос.

```py
from django.contrib.auth.mixins import LoginRequiredMixin

# Наследуем BirthdayCreateView от CreateView и от миксина LoginRequiredMixin:
class BirthdayCreateView(LoginRequiredMixin, CreateView):
    model = Birthday
    form_class = BirthdayForm 
```

При этом никаких изменений в маршрут вносить не нужно, он остаётся тем же, что и был:

```py
urlpatterns = [
    ...
    path('create/', views.BirthdayCreateView.as_view(), name='create'),
    ...
] 
```

Добавьте ко всем CBV, которые изменяют данные в базе (создание записи, редактирование, удаление) миксин `LoginRequiredMixin`.

***
## Работа с формами и пользователями

Анонимных пользователей в правах ограничили, теперь нужно разобраться с правами на редактирование и удаление объектов. 

Будет логично предоставить доступ к редактированию и удалению объектов только авторам — пользователям, создавшим эти объекты.

На книгах ставят имя автора, в титрах фильма — имя режиссёра; в каждом объекте модели `Birthday` должно храниться имя пользователя, создавшего его, — каждый объект `Birthday` должен быть связан с определённым объектом модели пользователя.

Создайте в модели `Birthday` новое поле:

```py
# birthday/models.py
from django.contrib.auth import get_user_model

# Да, именно так всегда и ссылаемся на модель пользователя!
User = get_user_model()


class Birthday(models.Model):
    ...
    author = models.ForeignKey(
        User, verbose_name='Автор записи', on_delete=models.CASCADE, null=True
    ) 
```

Обратите внимание: поле `author` может принимать нулевое значение (`null=True`), это позволит не указывать значение по умолчанию для уже существующих записей о днях рождения в БД.

![alt text](https://pictures.s3.yandex.net/resources/image_1703843741.png)

В настройках формы есть строка `fields = '__all__'` — «показывать в форме все поля, которые есть в модели». Поле «Автор», добавленное в модель, появилось и в HTML-форме, но для нашей задачи это не просто не нужно, но и вредно. 

Через форму можно «назначить» автора записи, выбрав любого из числа пользователей системы; получается, можно опубликовать запись от имени любого пользователя. 

Так не годится: задача состоит в том, чтобы вместе с данными из формы сохранялся объект именно того пользователя, который заполнил и отправил форму. 

Настройка `fields = '__all__'` подпортила картину. Надо исправлять ситуацию, а для этого надо убрать поле `author` из HTML-формы.

Перенастроим класс формы; вместо значения `__all__` можно явным образом перечислить все поля, которые должны отображаться в форме. 

А можно вместо `fields` поставить атрибут `exclude` и указать там поле `author`. Так и поступим: это короче.

```py
class BirthdayForm(forms.ModelForm):

    class Meta:
        model = Birthday
        exclude = ('author',)
        ... 
```

Теперь в HTML-форме не будет поля «Автор». Если сейчас отправить заполненную форму, то будет создан новый объект с пустым полем `author`. Для этого поля установлен аргумент `null=True`, следовательно, при отправке формы с пустым полем `author` ошибки не возникнет. Однако задача состоит в том, чтобы передать объект автора записи в объект формы; добавить объект автора в поле `author` нужно перед сохранением объекта формы.

Объект пользователя можно получить прямо из запроса, в котором отправлены данные формы: объект пользователя, отправившего запрос, доступен в свойстве `request.user`. Остаётся передать объект пользователя в поле объекта формы.

Документация [сообщает](https://docs.djangoproject.com/en/5.1/topics/class-based-views/generic-editing/#models-and-request-user), что присвоить значение нужному полю в CBV можно через переопределение метода валидации:

```py
class BirthdayCreateView(LoginRequiredMixin, CreateView):
    model = Birthday
    form_class = BirthdayForm

    def form_valid(self, form):
        # Присвоить полю author объект пользователя из запроса.
        form.instance.author = self.request.user
        # Продолжить валидацию, описанную в форме.
        return super().form_valid(form) 
```

Если в проекте применяются не CBV, а view-функции — процесс выглядит иначе:

1. При обработке формы создаётся объект модели `Birthday`, но не сохраняется в БД (если сохранить его сразу — в нём не будет данных об авторе). 
Создание объекта модели без его сохранения выполняется с помощью метода `form.save()` с аргументом `commit=False`.

2. После этого полю объекта `author` присваивается нужное значение.

3. Теперь нужно сохранить объект модели в БД: у объектов моделей тоже есть метод `save()`. Сохранять нужно именно объект модели, а не форму — ведь значение поля `author` записано именно в объект модели.

```py
def birthday(request):
    form = BirthdayForm(request.POST or None)
    context = {'form': form}
    if form.is_valid():
         instance = form.save(commit=False)
         instance.author = request.user
         instance.save()
```

Готово: пользователь привязан к объекту модели `Birthday`, можно проверять авторство объекта.

***
## Проверка авторства объекта

Итак, с каждым объектом модели `Birthday` теперь связан определённый пользователь, автор записи. Но это никак не мешает другому зарегистрированному пользователю взять и отредактировать чужую запись, а то и удалить её.

Сформулируем задачу: доступ к страницам редактирования и удаления записей должен быть только у автора записи. Если же на страницу пытается зайти любой другой пользователь — с ним можно поступить по-разному: 

* можно переадресовать на какую-то другую страницу проекта;

* можно вернуть ему страницу ошибки.

Главное — не пустить постороннего пользователя на запретную для него страницу.

Можно вернуть пользователю 403 ошибку — «такая страница существует, но у вас недостаточно прав для её посещения»; но из этого сообщения пользователь поймёт, что эта страница действительно существует, и может продолжить попытки проникнуть на неё. 

А можно сделать вид, что этой страницы и вовсе нет, — и вернуть пользователю ошибку 404. 

В зависимости от того, какие представления применены в проекте — view-функции или CBV, — проверку авторства нужно выполнять по-разному. 

При работе с view-функциями объект модели обычно получают с помощью функции-«шортката» `get_object_or_404()`. Для проверки авторизации в эту функцию добавляется фильтр, который сравнивает значение поля `author` модели с объектом пользователя из запроса.

```py
def edit_birthday(request, pk):
    # При поиске объекта дополнительно указываем текущего пользователя.
    instance = get_object_or_404(Birthday, pk=pk, author=request.user) 
```

Если запрос к странице редактирования или удаления объекта сделал не тот пользователь, который указан в поле `author` запрошенного объекта модели, то в ответ на запрос вернётся ошибка 404.

Если нужно вернуть другую ошибку (например, 403) или переадресовать пользователя на другую страницу, то сначала нужно получить объект, а потом проверить его авторство.

```py
# Импортируем класс ошибки (в данном случае - 403).
from django.core.exceptions import PermissionDenied        


def edit_birthday(request, pk):
    # Получаем нужный объект.
    instance = get_object_or_404(Birthday, pk=pk)
    # Проверяем, кто автор объекта.
    if instance.author != request.user:
        # Здесь может быть как вызов ошибки, так и редирект на нужную страницу.
        raise PermissionDenied 
```

***
При работе с CBV проверить права доступа можно разными способами. Рассмотрим один из них, рекомендованный документацией Django. 

К CBV добавляется специальный миксин [UserPassesTestMixin](https://docs.djangoproject.com/en/3.2/topics/auth/default/#django.contrib.auth.mixins.UserPassesTestMixin) (дословно «Пользователь проходит тест»), и описывается метод с предустановленным названием `test_func()`. 

В методе `test_func()` нужно написать своего рода тест, который пройдёт пользователь, запрашивающий доступ. Метод должен вернуть `True` (тест пройден) или `False` (тест провален), и в зависимости от этого доступ либо будет предоставлен, либо будет вызвана ошибка 403. 

Тест может быть любым: можно проверить, что логин пользователя — `'Stac_basov'`; или что сейчас полдень; или (как в примере ниже) что объект пользователя, запросившего доступ к странице, совпадает с объектом, сохранённым в поле `author` запрошенного объекта модели.

Класс `UserPassesTestMixin` унаследован от [AccessMixin](https://docs.djangoproject.com/en/3.2/topics/auth/default/#django.contrib.auth.mixins.AccessMixin), который по умолчанию переадресует анонимных пользователей на страницу логина. Поэтому при использовании `UserPassesTestMixin` миксин `LoginRequiredMixin` можно не использовать: он будет избыточным.

Посмотрим на примере:

```py
from django.contrib.auth.mixins import UserPassesTestMixin


# Добавляем миксин для тестирования пользователей, обращающихся к объекту.
class BirthdayUpdateView(UserPassesTestMixin, UpdateView):
    model = Birthday
    form_class = BirthdayForm

    # Определяем метод test_func() для миксина UserPassesTestMixin:
    def test_func(self):
        # Получаем текущий объект.
        object = self.get_object()
        # Метод вернёт True или False. 
        # Если пользователь - автор объекта, то тест будет пройден.
        # Если нет, то будет вызвана ошибка 403.
        return object.author == self.request.user 
```

Такую проверку надо разместить во всех CBV, где нужна проверка авторства. Выглядит громоздко: помимо того, что при объявлении CBV надо добавить миксин `UserPassesTestMixin`, в каждый класс придётся добавлять одно и то же описание метода `test_func()`. 

В такой ситуации гораздо выгоднее написать собственный миксин, унаследованный от `UserPassesTestMixin`:

```py
class OnlyAuthorMixin(UserPassesTestMixin):

    def test_func(self):
        object = self.get_object()
        return object.author == self.request.user 
```

Теперь этот миксин можно добавлять ко всем CBV, где требуется проверять авторство объекта — например, в CBV страниц редактирования или удаления записей.

***
## Разграничение прав в шаблоне

Любой пользователь проекта на странице */birthday/list/* под каждой записью увидит ссылки на редактирование и удаление записи. Воспользоваться этими ссылками не сможет никто, кроме автора записи. Будет гуманно не провоцировать остальных пользователей на ошибку и просто скрыть эти ссылки ото всех, кроме автора записи. 

В HTML-шаблоне доступен как объект пользователя, просматривающего страницу — `user`, так и объект автора записи — `birthday.author`. Ссылки на редактирование и удаление записи следует показывать лишь в том случае, если `birthday.author == user`.

Добавим в шаблон проверку этого условия — и зарегистрированные пользователи будут видеть эти ссылки только под своими записями, а анонимным посетителям эти ссылки и не будут видны вообще никогда.

```html
{% if birthday.author == user %}
  <div>
    <a href="{% url 'birthday:edit' birthday.id %}">Изменить запись</a> | <a href="{% url 'birthday:delete' birthday.id %}">Удалить запись</a>
  </div>
{% endif %} 
```

> Но прежде чем переходить к следующему уроку, сохраните в закладки [шпаргалку](https://code.s3.yandex.net/Python-dev/cheatsheets/035-django-polzovateli-shpora/035-django-polzovateli-shpora.html) по теме «Пользователи в Django». Впереди вас ещё ждут задачи, где она вам пригодится.